<a href="https://colab.research.google.com/github/cagBRT/timeSeries/blob/main/10c_MultiHeaded_CNN_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **MultiHeaded CNN Models**

Just as we did with the multiHeaded MLPs, we can create multiheaded CNNs

Multiheaded models can often be more flexible and/or more accurate than a single headed model.

**The split_Sequences function**

In [ ]:
def split_sequences(sequences, n_steps): 
  X, y = list(), list()
  for i in range(len(sequences)):
    # find the end of this pattern
    end_ix = i + n_steps
    # check if we are beyond the dataset
    if end_ix > len(sequences): 
      break
    # gather input and output parts of the pattern
    seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1] 
    X.append(seq_x)
    y.append(seq_y)
  return array(X), array(y)

**The libraries**

In [ ]:
from numpy import array
from numpy import hstack
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D 
from keras.layers.merge import concatenate
from keras.utils.vis_utils import plot_model

**Creating a time series**

In [ ]:
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+(2*in_seq2[i]) for i in range(len(in_seq1))])
# convert to [rows, columns] structure 
in_seq1 = in_seq1.reshape((len(in_seq1), 1)) 
in_seq2 = in_seq2.reshape((len(in_seq2), 1)) 
out_seq = out_seq.reshape((len(out_seq), 1)) 
# horizontally stack columns
dataset = hstack((in_seq1, in_seq2, out_seq))

In [ ]:
dataset

In [ ]:
# choose a number of time steps
n_steps = 3
# convert into input/output
X, y = split_sequences(dataset, n_steps)
# one time series per head
n_features = 1
X1 = X[:, :, 0].reshape(X.shape[0], X.shape[1], n_features) 
X2 = X[:, :, 1].reshape(X.shape[0], X.shape[1], n_features) 

**First head**

In [ ]:
head1 = Input(shape=(n_steps, n_features)) 
cnn1 = Conv1D(20, 2, activation='relu')(head1) 
cnn1 = MaxPooling1D()(cnn1)
cnn1 = Flatten()(cnn1)

**Second head**

In [ ]:
head2 = Input(shape=(n_steps, n_features)) 
cnn2 = Conv1D(20, 2, activation='relu')(head2) 
cnn2 = MaxPooling1D()(cnn2)
cnn2 = Flatten()(cnn2)

**Merge the two heads into one model**

In [ ]:
merge = concatenate([cnn1, cnn2])
dense = Dense(20, activation='relu')(merge)
output = Dense(1)(dense)
model = Model(inputs=[head1, head2], outputs=output) 

In [ ]:
model.compile(optimizer='adam', loss='mse')

**Plot the model architecture**

In [ ]:
plot_model(model, show_shapes=True, show_layer_names=True)

**Train the model**

In [ ]:
model.fit([X1, X2], y, epochs=100, verbose=0)

**Make a prediction**

In [ ]:
x_input = array([[80, 85], [90, 95], [100, 105]])
x1 = x_input[:, 0].reshape((1, n_steps, n_features))
x2 = x_input[:, 1].reshape((1, n_steps, n_features))
yhat = model.predict([x1, x2], verbose=0)
print(yhat)

**Assignment:**<br>
Change the number of epochs and neurons in the model to improve the prediction. <br>
The number of neurons in the two input models should be the same